In [1]:
import scanpy as sc
import pandas as pd
import anndata as ad
import anndata2ri
import os
from os.path import join
import scipy.sparse as sp
import numpy as np

from hlca_v2.ingestion_utils import get_gspread_df, ValidationWorkflow, AnnDataMerger

GSPREAD_JSON = "/home/icb/raphael.kfuri-rubens/data/hlca_v2/google_sheets_api/hlca-v2-8d5fea10d8f3.json"

During startup - Warning messages:
1: package ‘utils’ in options("defaultPackages") was not found 
2: package ‘stats’ in options("defaultPackages") was not found 
R[write to console]: Error in .Primitive("as.environment")("package:utils") : 
  no item called "package:utils" on the search list



RRuntimeError: Error in .Primitive("as.environment")("package:utils") : 
  no item called "package:utils" on the search list


In [ ]:
DATASET_ID = "Nawijn_unpubl"
H5AD_PATH = f"/home/icb/raphael.kfuri-rubens/data/hlca_v2/{DATASET_ID}/{DATASET_ID}_2024_ensembleIDs.h5ad"
OUTPUT_PATH_PREREVISION = '/home/icb/raphael.kfuri-rubens/data/hlca_v2/HLCA_V2_CORE/adata_prerevision'
OUTPUT_PATH_POSTREVISION = '/home/icb/raphael.kfuri-rubens/data/hlca_v2/HLCA_V2_CORE/adata_postrevision'

In [4]:
# Name constants
AUTHOR_CELL_TYPE_L0 = 'author_cell_type_level_0'
AUTHOR_CELL_TYPE_L1 = 'author_cell_type_level_1'
AUTHOR_CELL_TYPE_L2 = 'author_cell_type_level_2'
AUTHOR_CELL_TYPE_L3 = 'author_cell_type_level_3'

CELL_TYPE_ONTOLOGY_ID_L3 = 'cell_type_ontology_term_id_level_3'

# Finest grained annotation will be generic dataset cell type
AUTHOR_CELL_TYPE = 'author_cell_type'
CELL_TYPE_ONTOLOGY_ID = 'cell_type_ontology_term_id'
MARKER_GENES = 'author_cell_type_markers'

# Load data

In [5]:
adata = sc.read_h5ad(H5AD_PATH)
obs = get_gspread_df(GSPREAD_JSON, DATASET_ID, "tier_1", "obs")
uns = get_gspread_df(GSPREAD_JSON, DATASET_ID, "tier_1", "uns")

# Validate obs and uns from Tier 1 Metadata Template

In [6]:
val_workflow = ValidationWorkflow(
    input = uns,
    axis = 'uns'
)

validated_uns = val_workflow.init_workflow()
validated_uns

Validation workflow successful. Returning UNS dataframe.

,title,study_PI,batch_condition,default_embedding,unpublished,comments
0,All--cells from adult human lung from low-air...,Martijn Nawijn,donorID,X_scANVI,"consented for release, protected under embargo",-


In [7]:
val_workflow = ValidationWorkflow(
    input = obs,
    axis = 'obs'
)

validated_obs = val_workflow.init_workflow()
validated_obs

Validation workflow successful. Returning OBS dataframe.

,sample_ID,donor_id,protocol_URL,institute,sample_collection_site,sample_collection_relative_time_point,library_ID,library_ID_repository,author_batch_notes,organism_ontology_term_id,...,sequenced_fragment,sequencing_platform,is_primary_data,reference_genome,gene_annotation_version,alignment_software,intron_inclusion,disease_ontology_term_id,self_reported_ethnicity_ontology_term_id,development_stage_ontology_term_id
0,S02,GRO-02,N/A,University Medical Center Groningen,UMCG_EXPIRE,S2_day1,N/A,N/A,Batch run by different personnel on different ...,NCBITaxon:9606,...,3 prime tag,EFO_0008563,true,GRCh38,v98,cellranger-6.1.2,no,PATO:0000461,unknown,HsapDv:0000241
1,S03,GRO-03,N/A,University Medical Center Groningen,UMCG_EXPIRE,S3_day1,N/A,N/A,Batch run by different personnel on different ...,NCBITaxon:9606,...,3 prime tag,EFO_0008563,true,GRCh38,v98,cellranger-6.1.2,no,PATO:0000461,unknown,HsapDv:0000240
2,S04,GRO-04,N/A,University Medical Center Groningen,UMCG_EXPIRE,S4_day1,N/A,N/A,Batch run by different personnel on different ...,NCBITaxon:9606,...,3 prime tag,EFO_0008563,true,GRCh38,v98,cellranger-6.1.2,no,PATO:0000461,unknown,HsapDv:0000240
3,S05,GRO-05,N/A,University Medical Center Groningen,UMCG_EXPIRE,S5_day1,N/A,N/A,Batch run by different personnel on different ...,NCBITaxon:9606,...,3 prime tag,EFO_0008563,true,GRCh38,v98,cellranger-6.1.2,no,PATO:0000461,unknown,HsapDv:0000239
4,S06,GRO-06,N/A,University Medical Center Groningen,UMCG_EXPIRE,S07_day1,N/A,N/A,Batch run by different personnel on different ...,NCBITaxon:9606,...,3 prime tag,EFO_0008563,true,GRCh38,v98,cellranger-6.1.2,no,PATO:0000461,unknown,HsapDv:0000240
5,S07,GRO-07,N/A,University Medical Center Groningen,UMCG_EXPIRE,S7_day1,N/A,N/A,Batch run by different personnel on different ...,NCBITaxon:9606,...,3 prime tag,EFO_0008563,true,GRCh38,v98,cellranger-6.1.2,no,PATO:0000461,unknown,HsapDv:0000241
6,S08,GRO-08,N/A,University Medical Center Groningen,UMCG_EXPIRE,S8_day1,N/A,N/A,Batch run by different personnel on different ...,NCBITaxon:9606,...,3 prime tag,EFO_0008563,true,GRCh38,v98,cellranger-6.1.2,no,PATO:0000461,unknown,HsapDv:0000240
7,S09,GRO-09,N/A,University Medical Center Groningen,UMCG_EXPIRE,S9_day1,N/A,N/A,Batch run by different personnel on different ...,NCBITaxon:9606,...,3 prime tag,EFO_0008563,true,GRCh38,v98,cellranger-6.1.2,no,PATO:0000461,unknown,HsapDv:0000240
8,S10,GRO-10,N/A,University Medical Center Groningen,UMCG_EXPIRE,S10_day1,N/A,N/A,Batch run by different personnel on different ...,NCBITaxon:9606,...,3 prime tag,EFO_0008563,true,GRCh38,v98,cellranger-6.1.2,no,PATO:0000461,unknown,HsapDv:0000240
9,S11,GRO-11,N/A,University Medical Center Groningen,UMCG_EXPIRE,S11_day1,N/A,N/A,Batch run by different personnel on different ...,NCBITaxon:9606,...,3 prime tag,EFO_0008563,true,GRCh38,v98,cellranger-6.1.2,no,PATO:0000461,unknown,HsapDv:0000240


# Validate obs and uns from adata

In [8]:
adata

AnnData object with n_obs × n_vars = 147242 × 36601
    obs: 'donorID', 'sampleID', 'sex', '10x_v2_or_v3', 'Cell_onto(CL)', 'condition', 'sample_type', 'doublet_score', 'Mylevel1_annot', 'Mylevel2_annot', 'Mylevel3_annot', 'Mylevel4_annot', 'n_genes', 'n_counts', 'n_genes_by_counts', 'total_counts', 'total_counts_mt', 'pct_counts_mt'
    var: 'gene_names', 'ensembleID'
    uns: 'Mylevel1_annot_colors', 'Mylevel4_annot_colors', '_scvi_manager_uuid', '_scvi_uuid', 'hvg', 'log1p', 'pca', 'sampleID_colors', 'sample_type_colors', 'scANVI', 'umap'
    obsm: 'Unintegrated', 'X-Scanorama', 'X_Harmony', 'X_pca', 'X_scANVI', 'X_scGen', 'X_scVI', 'X_umap'
    obsp: 'scANVI_connectivities', 'scANVI_distances'

In [9]:
merger = AnnDataMerger(
    adata = adata,
    uns_df = uns
)

adata = merger.add_uns_metadata()

adata

AnnData object with n_obs × n_vars = 147242 × 36601
    obs: 'donorID', 'sampleID', 'sex', '10x_v2_or_v3', 'Cell_onto(CL)', 'condition', 'sample_type', 'doublet_score', 'Mylevel1_annot', 'Mylevel2_annot', 'Mylevel3_annot', 'Mylevel4_annot', 'n_genes', 'n_counts', 'n_genes_by_counts', 'total_counts', 'total_counts_mt', 'pct_counts_mt'
    var: 'gene_names', 'ensembleID'
    uns: 'Mylevel1_annot_colors', 'Mylevel4_annot_colors', '_scvi_manager_uuid', '_scvi_uuid', 'hvg', 'log1p', 'pca', 'sampleID_colors', 'sample_type_colors', 'scANVI', 'umap', 'title', 'study_PI', 'batch_condition', 'default_embedding', 'unpublished', 'comments'
    obsm: 'Unintegrated', 'X-Scanorama', 'X_Harmony', 'X_pca', 'X_scANVI', 'X_scGen', 'X_scVI', 'X_umap'
    obsp: 'scANVI_connectivities', 'scANVI_distances'

In [10]:
obs_ids = set(obs['sample_ID'].value_counts().index)
adata_ids = set(adata.obs['sampleID'].value_counts().index)
symmetric_difference = obs_ids.symmetric_difference(adata_ids)
symmetric_difference

set()

In [11]:
# Merge obs and uns
merger = AnnDataMerger(
    adata = adata,
    obs_df = obs
)

adata = merger.add_obs_metadata(
    adata_col = 'sampleID',
    df_col = 'sample_ID',
    skip = None,
)

adata.obs['sample_ID'] = adata.obs['sampleID']

adata.obs

,donorID,sampleID,sex,10x_v2_or_v3,Cell_onto(CL),condition,sample_type,doublet_score,Mylevel1_annot,Mylevel2_annot,...,sequencing_platform,is_primary_data,reference_genome,gene_annotation_version,alignment_software,intron_inclusion,disease_ontology_term_id,self_reported_ethnicity_ontology_term_id,development_stage_ontology_term_id,sample_ID
AAACCTGAGGAGTTGC-1_GRO-12,GRO-12,S12,Male,V2,CL:0005012,Healthy,br_biopsy,0.072319,Epithelial,Airway epithelium,...,EFO_0008563,true,GRCh38,v98,cellranger-6.1.2,no,PATO:0000461,unknown,HsapDv:0000240,S12
AAAGATGAGGCGCTCT-1_GRO-12,GRO-12,S12,Male,V2,CL:0005012,Healthy,br_biopsy,0.089035,Epithelial,Airway epithelium,...,EFO_0008563,true,GRCh38,v98,cellranger-6.1.2,no,PATO:0000461,unknown,HsapDv:0000240,S12
AAAGCAACAGTACACT-1_GRO-12,GRO-12,S12,Male,V2,CL:0005012,Healthy,br_biopsy,0.127660,Epithelial,Airway epithelium,...,EFO_0008563,true,GRCh38,v98,cellranger-6.1.2,no,PATO:0000461,unknown,HsapDv:0000240,S12
AAAGCAATCCTAGTGA-1_GRO-12,GRO-12,S12,Male,V2,CL:0005012,Healthy,br_biopsy,0.127660,Epithelial,Airway epithelium,...,EFO_0008563,true,GRCh38,v98,cellranger-6.1.2,no,PATO:0000461,unknown,HsapDv:0000240,S12
AAAGTAGGTTGTCTTT-1_GRO-12,GRO-12,S12,Male,V2,CL:0005012,Healthy,br_biopsy,0.096457,Epithelial,Airway epithelium,...,EFO_0008563,true,GRCh38,v98,cellranger-6.1.2,no,PATO:0000461,unknown,HsapDv:0000240,S12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
AGGTGTTCAAATACGA-1_GRO-27,GRO-27,S27,female,V3.1,CL:0000165,Healthy,br_biopsy,0.103030,Epithelial,Airway epithelium,...,EFO_0008637,true,GRCh38,v98,cellranger-6.1.2,no,PATO:0000461,unknown,HsapDv:0000239,S27
ATCTCTAGTACAATAG-1_GRO-27,GRO-27,S27,female,V3.1,CL:0000165,Healthy,br_biopsy,0.142140,Epithelial,Airway epithelium,...,EFO_0008637,true,GRCh38,v98,cellranger-6.1.2,no,PATO:0000461,unknown,HsapDv:0000239,S27
ATGAAAGCAGGTGACA-1_GRO-27,GRO-27,S27,female,V3.1,CL:0000165,Healthy,br_biopsy,0.126348,Epithelial,Airway epithelium,...,EFO_0008637,true,GRCh38,v98,cellranger-6.1.2,no,PATO:0000461,unknown,HsapDv:0000239,S27
GGGATGAAGACCAGAC-1_GRO-27,GRO-27,S27,female,V3.1,CL:0000165,Healthy,br_biopsy,0.121622,Epithelial,Airway epithelium,...,EFO_0008637,true,GRCh38,v98,cellranger-6.1.2,no,PATO:0000461,unknown,HsapDv:0000239,S27


# Add author cell type markers to UNS

In [12]:
adata.obs.columns

Index(['donorID', 'sampleID', 'sex', '10x_v2_or_v3', 'Cell_onto(CL)',
       'condition', 'sample_type', 'doublet_score', 'Mylevel1_annot',
       'Mylevel2_annot', 'Mylevel3_annot', 'Mylevel4_annot', 'n_genes',
       'n_counts', 'n_genes_by_counts', 'total_counts', 'total_counts_mt',
       'pct_counts_mt', 'donor_id', 'protocol_URL', 'institute',
       'sample_collection_site', 'sample_collection_relative_time_point',
       'library_ID', 'library_ID_repository', 'author_batch_notes',
       'organism_ontology_term_id', 'manner_of_death', 'sample_source',
       'sex_ontology_term_id', 'sample_collection_method', 'tissue_type',
       'sampled_site_condition', 'tissue_ontology_term_id', 'tissue_free_text',
       'sample_preservation_method', 'suspension_type', 'cell_enrichment',
       'cell_viability_percentage', 'cell_number_loaded',
       'sample_collection_year', 'assay_ontology_term_id',
       'library_preparation_batch', 'library_sequencing_run',
       'sequenced_fragment

In [13]:
marker_genes_dict = {
    "Club": ['CYP2F1','SCGB3A1','BPIFB1'],
    "Goblet": ['ALDH1A3','CEACAM5','MUC5B','MUC5AC'],
    "Basal": ["KRT5","S100A2"],
    "Basal resting cells": ["KRT15","KRT17","TP63"],
    "Cycling Basal": ["TOP2A","MKI67"],
    "Suprabasal": ["SERPINB3"],
    "Hillock-like": ["KRT13","KRT4","DSG3","TNS4",'KRT6A'],
    "Precursor hillock-like": ["KRT13","DSG3","TNS4",'KRT6A','KRT14'],
    "RBCs": ["HBB", "HBA1","HBA2"],
    #"Dying cells": ["MALAT1","NEAT1","pct_counts_mt"],
    "Deuterosomal": ["CDC20B","E2F7"],
    "Multiciliated": ["FOXJ1"],
    "Multiciliated (HLCA)": ["RSPH1","C20orf85"],
    "SMG": ["LYZ","AZGP1"],
    #"SMG - Duct": ["MGST1"],
    "SMG - Serous": ["S100A1",'PRB3'],
    "SMG - Mucous": ["BPIFB2","MUC5B","NKX3-1"],
    "Rare": ["FOXI1","CFTR","BSND","IGF1",'CLCNKB', "SLC6A17","DAB1"],
    "RBCs": ["HBB", "HBA1","HBA2"],
    "B cells": ["MS4A1"],
    #"Plasma cells": ["TNFRSF17"],
    #"T cells proliferating": ["CENPW", "TK1", "MKI67"],
    "CD4 T cells": ["CD40LG", "CD4"], # poss. lowly expressed, non-unique, non-universal
    "CD8 T cells": ["CD8A", "CD8B", "TRGC2"], # poss lowly expressed, non unique
    "NK cells": ["GNLY", "SPON2", "FCGR3A"],
    "Plasmacytoid DCs": ["SCT",  "LILRA4"],
    "DC1": ["XCR1", "CLEC9A", "CLNK"],
    "DC2": ["CLEC10A", "CD1E"],
    "Migratory DCs": ["LAD1", "CCL19"],
    "Alveolar macrophages": [ "MARCO", "FABP4"],
    "Alveolar mph CCL3+": ["CCL20"],
    "Alveolar Mph MT-positive": ["MT1M", "CCL18"],
    "Alveolar Mph proliferating": ["TYMS", "CENPW"],
    "Interstitial Mph perivascular": ["F13A1"],
    "Monocyte derived Mph": ["SPP1"],
    "Classical monocytes": ["S100A12", "FCN1", "RNASE2"],
    "Non-classical monocyte": ["LILRA5", "FCN1"],
    "Mast cell": ["MS4A2", "SLC18A2", "RGS13"],
    "Hematopoietic stem cell": ["CRHBP", "SERPINE2"]
}

In [14]:
# convert marker_genes_dict to df
marker_genes_df = pd.DataFrame(marker_genes_dict.items(), columns=[AUTHOR_CELL_TYPE, MARKER_GENES])
marker_genes_df[MARKER_GENES] = marker_genes_df[MARKER_GENES].apply(lambda x: ', '.join(x))

# replace , by ;
marker_genes_df[MARKER_GENES] = marker_genes_df[MARKER_GENES].str.replace(',', ';')
marker_genes_df

,author_cell_type,author_cell_type_markers
0,Club,CYP2F1; SCGB3A1; BPIFB1
1,Goblet,ALDH1A3; CEACAM5; MUC5B; MUC5AC
2,Basal,KRT5; S100A2
3,Basal resting cells,KRT15; KRT17; TP63
4,Cycling Basal,TOP2A; MKI67
5,Suprabasal,SERPINB3
6,Hillock-like,KRT13; KRT4; DSG3; TNS4; KRT6A
7,Precursor hillock-like,KRT13; DSG3; TNS4; KRT6A; KRT14
8,RBCs,HBB; HBA1; HBA2
9,Deuterosomal,CDC20B; E2F7


In [15]:
adata.uns[MARKER_GENES] = marker_genes_df

# Check author cell type annotations and Cell Ontology IDs

In [16]:
adata.obs.columns

Index(['donorID', 'sampleID', 'sex', '10x_v2_or_v3', 'Cell_onto(CL)',
       'condition', 'sample_type', 'doublet_score', 'Mylevel1_annot',
       'Mylevel2_annot', 'Mylevel3_annot', 'Mylevel4_annot', 'n_genes',
       'n_counts', 'n_genes_by_counts', 'total_counts', 'total_counts_mt',
       'pct_counts_mt', 'donor_id', 'protocol_URL', 'institute',
       'sample_collection_site', 'sample_collection_relative_time_point',
       'library_ID', 'library_ID_repository', 'author_batch_notes',
       'organism_ontology_term_id', 'manner_of_death', 'sample_source',
       'sex_ontology_term_id', 'sample_collection_method', 'tissue_type',
       'sampled_site_condition', 'tissue_ontology_term_id', 'tissue_free_text',
       'sample_preservation_method', 'suspension_type', 'cell_enrichment',
       'cell_viability_percentage', 'cell_number_loaded',
       'sample_collection_year', 'assay_ontology_term_id',
       'library_preparation_batch', 'library_sequencing_run',
       'sequenced_fragment

In [17]:
annotations = adata.obs[['Mylevel1_annot','Mylevel2_annot', 'Mylevel3_annot', 'Mylevel4_annot', 'Cell_onto(CL)']].drop_duplicates(subset='Mylevel4_annot').reset_index(drop=True)
annotations

,Mylevel1_annot,Mylevel2_annot,Mylevel3_annot,Mylevel4_annot,Cell_onto(CL)
0,Epithelial,Airway epithelium,Multiciliated lineage,Multiciliated cells,CL:0005012
1,Epithelial,Airway epithelium,Basal,Suprabasal cells,CL:4033048
2,Immune,Myeloid,Dendritic cells,Monocytes-derived DCs,none
3,Epithelial,Airway epithelium,Multiciliated lineage,Mucous ciliated cells,none
4,Epithelial,Airway epithelium,Secretory,Club cells,CL:0000158
5,Immune,Myeloid,Macrophages,Monocyte derived macrophages,none
6,Immune,Myeloid,Monocytes,Classical monocytes,CL:0000860
7,Immune,Lymphoid,T cell lineage,CD4 T cells,CL:4033038
8,Epithelial,Airway epithelium,Basal,Cycling cells,none
9,Immune,Myeloid,Dendritic cells,DC2,none


In [18]:
annotations[MARKER_GENES] = annotations['Mylevel4_annot'].map(marker_genes_df.set_index(AUTHOR_CELL_TYPE)[MARKER_GENES])
annotations

,Mylevel1_annot,Mylevel2_annot,Mylevel3_annot,Mylevel4_annot,Cell_onto(CL),author_cell_type_markers
0,Epithelial,Airway epithelium,Multiciliated lineage,Multiciliated cells,CL:0005012,NaN
1,Epithelial,Airway epithelium,Basal,Suprabasal cells,CL:4033048,NaN
2,Immune,Myeloid,Dendritic cells,Monocytes-derived DCs,none,NaN
3,Epithelial,Airway epithelium,Multiciliated lineage,Mucous ciliated cells,none,NaN
4,Epithelial,Airway epithelium,Secretory,Club cells,CL:0000158,NaN
5,Immune,Myeloid,Macrophages,Monocyte derived macrophages,none,NaN
6,Immune,Myeloid,Monocytes,Classical monocytes,CL:0000860,S100A12; FCN1; RNASE2
7,Immune,Lymphoid,T cell lineage,CD4 T cells,CL:4033038,CD40LG; CD4
8,Epithelial,Airway epithelium,Basal,Cycling cells,none,NaN
9,Immune,Myeloid,Dendritic cells,DC2,none,CLEC10A; CD1E


In [19]:
adata.obs[AUTHOR_CELL_TYPE] = adata.obs['Mylevel4_annot']
adata.obs[AUTHOR_CELL_TYPE].value_counts()

Basal resting cells              31227
Suprabasal cells                 24875
Multiciliated cells              22147
CD8 T cells                      20258
Goblet cells                      7152
3_EC venous                       6328
Club cells                        4889
NK cells                          4026
Mucous ciliated cells             3529
3_Fibroblasts                     3144
B cells                           2688
CD4 T cells                       2507
DC2                               1919
Classical monocytes               1744
Monocyte derived macrophages      1400
3_Mast_cells                      1396
3_EC arterial                     1023
2_Smooth muscle                    932
Progenitor hillock-like cells      929
Plasma cells                       882
Monocytes-derived DCs              804
Cycling cells                      754
SMG - Serous cells                 682
SMG - Duct cells                   422
Hillock-like cells                 297
3_Macrophages            

In [20]:
adata.obs[CELL_TYPE_ONTOLOGY_ID] = adata.obs['Cell_onto(CL)']
adata.obs[CELL_TYPE_ONTOLOGY_ID].value_counts()

none          45568
CL:4033048    24875
CL:0005012    22147
CL:4033039    20258
CL:0000160     7152
CL:0002543     6328
CL:0000158     4889
CL:0000623     4026
CL:0002553     3144
CL:4033038     2507
CL:0000860     1744
CL:0000097     1396
CL:1000413     1023
CL:0000786      882
CL:4030024      297
CL:1001603      215
CL:0005006      204
CL:0000875      184
CL:0001058      153
CL:4033044      144
CL:0009115       88
CL:0000165       18
Name: cell_type_ontology_term_id, dtype: int64

In [21]:
adata.obs[AUTHOR_CELL_TYPE_L0] = adata.obs['Mylevel1_annot']
adata.obs[AUTHOR_CELL_TYPE_L1] = adata.obs['Mylevel2_annot']
adata.obs[AUTHOR_CELL_TYPE_L2] = adata.obs['Mylevel3_annot']
adata.obs[AUTHOR_CELL_TYPE_L3] = adata.obs['Mylevel4_annot']

adata.obs[CELL_TYPE_ONTOLOGY_ID_L3] = adata.obs['Cell_onto(CL)']

# Check whether ENSEMBL IDs in var

In [22]:
adata.var

,gene_names,ensembleID
A1BG,A1BG,ENSG00000121410
A1BG-AS1,A1BG-AS1,ENSG00000268895
A1CF,A1CF,ENSG00000148584
A2M,A2M,ENSG00000175899
A2M-AS1,A2M-AS1,ENSG00000245105
...,...,...
ZYG11B,ZYG11B,ENSG00000162378
ZYX,ZYX,ENSG00000159840
ZZEF1,ZZEF1,ENSG00000074755
hsa-mir-1253,hsa-mir-1253,ENSG00000272920


In [24]:
adata.var.index = adata.var['ensembleID']
adata.var.index.name = 'index'
adata.var.rename(columns={'gene_names': 'gene_symbol', 'ensembleID': 'ensembl_id'}, inplace=True)
adata.var

,gene_symbol,ensembl_id
index,,
ENSG00000121410,A1BG,ENSG00000121410
ENSG00000268895,A1BG-AS1,ENSG00000268895
ENSG00000148584,A1CF,ENSG00000148584
ENSG00000175899,A2M,ENSG00000175899
ENSG00000245105,A2M-AS1,ENSG00000245105
...,...,...
ENSG00000162378,ZYG11B,ENSG00000162378
ENSG00000159840,ZYX,ENSG00000159840
ENSG00000074755,ZZEF1,ENSG00000074755


# Check raw data

In [25]:
adata.X.toarray().max()

27821.0

In [26]:
adata.X = adata.X.astype(np.int64)

In [27]:
adata.raw = adata

In [28]:
adata.X

<147242x36601 sparse matrix of type '<class 'numpy.int64'>'
	with 256943696 stored elements in Compressed Sparse Row format>

In [29]:
adata.raw.X

<147242x36601 sparse matrix of type '<class 'numpy.int64'>'
	with 256943696 stored elements in Compressed Sparse Row format>

In [30]:
adata.X.toarray().max()

27821

In [31]:
adata.raw.X.toarray().max()

27821

## Validation result

### UNS Validation
- OK: Tier 1 UNS Google Sheet
- OK: Tier 1 UNS AnnData Object  same as Google Sheet

### OBS Validation
- OK: Tier 1 OBS Google Sheet
- OK: Tier 1 OBS AnnData Object same as Google Sheet


# Data Submission Status

- CHECK: Raw counts in X and in raw
- CHECK: Tier 1 Metadata in OBS
- CHECK: Cell Ontology IDs in OBS
- CHECK: Author cell type in OBS
- CHECK: Marker genes in UNS (however ontologies and marker genes could use some refinement)
- CHECK: ENSEMBL IDs and gene symbols in var

# Revisions:

DONE

In [ ]:
adata.write_h5ad(join(OUTPUT_PATH_PREREVISION, f"{DATASET_ID}.h5ad"))
adata.write_h5ad(join(OUTPUT_PATH_POSTREVISION, f"{DATASET_ID}.h5ad"))

# CellxGene ingestion prep

In [3]:
adata = sc.read_h5ad(join(OUTPUT_PATH_POSTREVISION, f"{DATASET_ID}.h5ad"))

/home/icb/raphael.kfuri-rubens/miniconda3/envs/hlcav2-env/lib/python3.10/site-packages/anndata/_core/anndata.py:121: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)
/home/icb/raphael.kfuri-rubens/miniconda3/envs/hlcav2-env/lib/python3.10/site-packages/anndata/_core/anndata.py:1830: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")


In [4]:
# check whether all obs index are unique
len(adata.obs) == len(adata.obs.index.unique())

True

In [6]:
adata.var.index = adata.var.index.astype(str)

In [7]:
adata.var_names_make_unique()

In [9]:
len(adata.var) == len(adata.var.index.unique())

True

In [8]:
adata.write_h5ad(join(OUTPUT_PATH_POSTREVISION, f"{DATASET_ID}.h5ad"))

In [ ]:
adata = sc.read_h5ad(join(OUTPUT_PATH_POSTREVISION, f"{DATASET_ID}.h5ad"))

In [2]:
adata = sc.read_h5ad("/ictstr01/home/icb/raphael.kfuri-rubens/data/hlca_v2/HLCA_V2_CORE/adata_postrevision_reduced/Nawijn_unpubl.h5ad")
adata

AnnData object with n_obs × n_vars = 147242 × 36601
    obs: 'sample_ID', 'donor_id', 'protocol_URL', 'institute', 'sample_collection_site', 'sample_collection_relative_time_point', 'library_ID', 'library_ID_repository', 'author_batch_notes', 'organism_ontology_term_id', 'manner_of_death', 'sample_source', 'sex_ontology_term_id', 'sample_collection_method', 'tissue_type', 'sampled_site_condition', 'tissue_ontology_term_id', 'tissue_free_text', 'sample_preservation_method', 'suspension_type', 'cell_enrichment', 'cell_viability_percentage', 'cell_number_loaded', 'sample_collection_year', 'assay_ontology_term_id', 'library_preparation_batch', 'library_sequencing_run', 'sequenced_fragment', 'sequencing_platform', 'is_primary_data', 'reference_genome', 'gene_annotation_version', 'alignment_software', 'intron_inclusion', 'disease_ontology_term_id', 'self_reported_ethnicity_ontology_term_id', 'development_stage_ontology_term_id', 'author_cell_type', 'cell_type_ontology_term_id', 'author_cell_

In [5]:
remove_uns = ['Mylevel1_annot_colors', 'Mylevel4_annot_colors', '_scvi_manager_uuid', '_scvi_uuid', 'hvg', 'log1p', 'pca', 'sampleID_colors', 'sample_type_colors', 'scANVI', 'umap']

for key in remove_uns:
    if key in adata.uns:
        del adata.uns[key]

del adata.obsm

del adata.obsp

adata

AnnData object with n_obs × n_vars = 147242 × 36601
    obs: 'sample_ID', 'donor_id', 'protocol_URL', 'institute', 'sample_collection_site', 'sample_collection_relative_time_point', 'library_ID', 'library_ID_repository', 'author_batch_notes', 'organism_ontology_term_id', 'manner_of_death', 'sample_source', 'sex_ontology_term_id', 'sample_collection_method', 'tissue_type', 'sampled_site_condition', 'tissue_ontology_term_id', 'tissue_free_text', 'sample_preservation_method', 'suspension_type', 'cell_enrichment', 'cell_viability_percentage', 'cell_number_loaded', 'sample_collection_year', 'assay_ontology_term_id', 'library_preparation_batch', 'library_sequencing_run', 'sequenced_fragment', 'sequencing_platform', 'is_primary_data', 'reference_genome', 'gene_annotation_version', 'alignment_software', 'intron_inclusion', 'disease_ontology_term_id', 'self_reported_ethnicity_ontology_term_id', 'development_stage_ontology_term_id', 'author_cell_type', 'cell_type_ontology_term_id', 'author_cell_

In [ ]:
adata.write("/ictstr01/home/icb/raphael.kfuri-rubens/data/hlca_v2/HLCA_V2_CORE/adata_postrevision_reduced/Nawijn_unpubl.h5ad")